In [58]:
import numpy as np


def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                                [w[2], 0, -w[0]],
                                [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

def BodyTwist(V_bmat):
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    w_b = w_b.reshape(-1,1)
    v_b = V_bmat[:3, 3]
    v_b = v_b.reshape(-1,1)
    output = np.vstack((w_b, v_b))
    return output
    
    

In [59]:
w1 = np.array([0, 0, 1])
v1 = np.array([0, 1, 0])
s1 = np.array([w1, v1])
s1 = s1.reshape(-1,1)
th = 0.5

T1 = screw_exp(s1, th)
T1

q = np.array([0, 0.166, -0.01])
M = np.vstack([np.hstack([np.eye(3), q.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])
M_ = T1 @ M
M_

A = Ad(M_)
A

B = BodyTwist(M_)
B




array([[0.0],
       [0.0],
       [0.479425538604203],
       [array([-0.20200208])],
       [array([0.62510424])],
       [array([-0.01])]], dtype=object)